# Implementation the best parameters settings for COMPAS, Adult Credit and FICO Credit score datasets

In [1]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

from src.util import mean_difference, get_list_of_seeds, mean
from src.feature_map import IdentityFeatureMap
from src.functions import cost_utility, cost_utility_gradient
from src.plotting import plot_mean, plot_median
from src.training import train
from src.training_evaluation import UTILITY, COVARIANCE_OF_DECISION_DP
from src.policy import LogisticPolicy
from src.distribution import AdultCreditDistribution, COMPASDistribution, FICODistribution
from src.optimization import ManualGradientPenaltyOptimizationTarget

unable to import 'smart_open.gcs', disabling that module


## Fairness Functions

In [3]:
def calc_benefit(decisions, ips_weights):
    if ips_weights is not None:
        decisions *= ips_weights

    return decisions


def calc_covariance(s, decisions, ips_weights):
    new_s = 1 - (2 * s)

    if ips_weights is not None:
        mu_s = np.mean(new_s * ips_weights, axis=0)
        d = decisions * ips_weights
    else:
        mu_s = np.mean(new_s, axis=0)
        d = decisions

    covariance = (new_s - mu_s) * d
    return covariance


def fairness_function_gradient(type, **fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]

    if type == "BD_DP" or type == "BD_EOP":
        result = calc_benefit(decisions, ips_weights)
    elif type == "COV_DP":
        result = calc_covariance(s, decisions, ips_weights)

    log_gradient = policy.log_policy_gradient(x, s)
    grad = log_gradient * result

    if type == "BD_DP":
        return mean_difference(grad, s)
    elif type == "COV_DP":
        return np.mean(grad, axis=0)
    elif type == "BD_EOP":
        y1_indices = np.where(y == 1)
        return mean_difference(grad[y1_indices], s[y1_indices])


def fairness_function(type, **fairness_kwargs):
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"] if "ips_weights" in fairness_kwargs else None

    if type == "BD_DP":
        benefit = calc_benefit(decisions, ips_weights)
        return mean_difference(benefit, s)
    elif type == "COV_DP":
        covariance = calc_covariance(s, decisions, ips_weights)
        return np.mean(covariance, axis=0)
    elif type == "BD_EOP":
        benefit = calc_benefit(decisions, ips_weights)
        y1_indices = np.where(y == 1)
        return mean_difference(benefit[y1_indices], s[y1_indices])

def no_fairness(**fairness_kwargs):
    return 0.0

def utility(**util_params):
    return cost_utility(cost_factor=0.5, **util_params)

def utility_gradient(**util_params):
    return cost_utility_gradient(cost_factor=0.5, **util_params)

def covariance_of_decision(**fairness_params):
    return fairness_function(
        type="COV_DP",
        **fairness_params)

def benefit_difference_dp(**fairness_params):
    return fairness_function(
        type="BD_DP",
        **fairness_params)

def benefit_difference_eop(**fairness_params):
    return fairness_function(
        type="BD_EOP",
        **fairness_params)

def covariance_of_decision_grad(**fairness_params):
    return fairness_function_gradient(
        type="COV_DP",
        **fairness_params)

def benefit_difference_dp_grad(**fairness_params):
    return fairness_function_gradient(
        type="BD_DP",
        **fairness_params)

def benefit_difference_eop_grad(**fairness_params):
    return fairness_function_gradient(
        type="BD_EOP",
        **fairness_params)

# COMPAS

In [4]:
bias = True
distribution = COMPASDistribution(bias=bias, test_percentage=0.2)
dim_theta = distribution.feature_dimension

training_parameters = {
    'model': LogisticPolicy(np.zeros(dim_theta), IdentityFeatureMap(dim_theta), False),
    'distribution': distribution,
    'parameter_optimization': {
        'time_steps':200,
        'epochs': 50,
        'batch_size':128,
        'learning_rate': 0.1,
        'learn_on_entire_history': False,
        'clip_weights': True
    },
    'data': {
        'num_train_samples': 4096,
        'num_test_samples': 1024
    },
    'evaluation': {
        UTILITY: {
            'measure_function': utility,
            'detailed': False
        },
        COVARIANCE_OF_DECISION_DP: {
            'measure_function': covariance_of_decision,
            'detailed': False
        }
    }
}

if os.path.isfile('./cluster_experiments/seeds.npz'):
    seeds = np.load('./cluster_experiments/seeds.npz')
    training_parameters['data']["training_seeds"] = seeds["train"]
    training_parameters['data']["test_seed"] = seeds["test"]
else:
    seeds = {}
    train_seeds = get_list_of_seeds(200)
    test_seeds = get_list_of_seeds(1)
    training_parameters['data']["training_seeds"] = train_seeds
    training_parameters['data']["test_seed"] = test_seeds
    np.savez('./cluster_experiments/seeds.npz', train=train_seeds, test=test_seeds)

### No Fariness

In [ ]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            utility,
                                            utility_gradient,
                                            no_fairness,
                                            no_fairness)

training_parameters["save_path"] = "../res/local_experiments/COMPAS_BEST/NO_FAIRNESS"
statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=30,
    asynchronous=True,
    fairness_rates=[0.0])

plot_mean(x_values=range(training_parameters["parameter_optimization"]["time_steps"] + 1),
          x_label="Time steps",
          x_scale="linear",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean_time.png".format(run_path))
plot_median(x_values=range(training_parameters["parameter_optimization"]["time_steps"] + 1),
          x_label="Time steps",
          x_scale="linear",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_median_time.png".format(run_path))


### Benefit Difference: DP

In [ ]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            utility,
                                            utility_gradient,
                                            benefit_difference_dp,
                                            benefit_difference_dp_grad)

# lambdas = np.linspace(0.0001, 0.001, endpoint=True, num=10).tolist()
# lambdas.extend(np.linspace(0.001, 0.01, endpoint=True, num=10).tolist())
# lambdas.extend(np.linspace(0.01, 0.1, endpoint=True, num=10).tolist())
# lambdas.extend(np.linspace(0.1, 1.0, endpoint=True, num=10).tolist())
# lambdas = np.sort(np.unique(np.array(lambdas, dtype=float)))
lambdas = np.geomspace(0.0001, 1.0, endpoint=True, num=10)

training_parameters["save_path"] = "../res/local_experiments/COMPAS_BEST/BD_DP"
statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=30,
    asynchronous=True,
    fairness_rates=lambdas)

plot_mean(x_values=lambdas,
          x_label="Penalty Constant",
          x_scale="log",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean.png".format(run_path))
plot_median(x_values=lambdas,
            x_label="Penalty Constant",
            x_scale="log",

          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
            file_path="{}/results_median.png".format(run_path))


### Benefit Difference: EOP

In [5]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            utility,
                                            utility_gradient,
                                            benefit_difference_eop,
                                            benefit_difference_eop_grad)

# lambdas = np.linspace(0.0001, 0.001, endpoint=True, num=10).tolist()
# lambdas.extend(np.linspace(0.001, 0.01, endpoint=True, num=10).tolist())
# lambdas.extend(np.linspace(0.01, 0.1, endpoint=True, num=10).tolist())
# lambdas.extend(np.linspace(0.1, 1.0, endpoint=True, num=10).tolist())
# lambdas = np.sort(np.unique(np.array(lambdas, dtype=float)))
lambdas = np.geomspace(0.0001, 1.0, endpoint=True, num=5)

training_parameters["save_path"] = "../res/local_experiments/COMPAS_BEST/BD_EOP"
statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=10,
    asynchronous=True,
    fairness_rates=lambdas)


plot_mean(x_values=lambdas,
          x_label="Penalty Constant",
          x_scale="log",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean.png".format(run_path))
plot_median(x_values=lambdas,
            x_label="Penalty Constant",
            x_scale="log",

          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
            file_path="{}/results_median.png".format(run_path))


## STARTED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.0001 ##
## ENDED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.0001 ##
## STARTED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.001 ##
## ENDED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.001 ##
## STARTED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.01 ##
## ENDED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.01 ##
## STARTED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.1 ##
## ENDED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 0.1 ##
## STARTED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 1.0 ##
## ENDED // LR = 0.1 // TS = 200 // E = 50 // BS = 128 // FR = 1.0 ##


# Adult Credit Data

In [ ]:
bias = True
distribution = AdultCreditDistribution(bias=bias, test_percentage=0.2)
dim_theta = distribution.feature_dimension

training_parameters = {
    'model': LogisticPolicy(np.zeros(dim_theta), IdentityFeatureMap(dim_theta), False),
    'distribution': distribution,
    'parameter_optimization': {
        'time_steps':200,
        'epochs': 50,
        'batch_size':512,
        'learning_rate': 0.05,
        'learn_on_entire_history': False,
        'clip_weights': True
    },
    'data': {
        'num_train_samples': 16384,
        'num_test_samples': 4096
    },
    'evaluation': {
        UTILITY: {
            'measure_function': lambda s, y, decisions : cost_utility(cost_factor=0.5,
                                                                              y=y,
                                                                              decisions=decisions),
            'detailed': False
        },
        COVARIANCE_OF_DECISION_DP: {
            'measure_function': lambda s, y, decisions : fairness_function(
                type="COV_DP",
                x=None,
                s=s,
                y=y,
                decisions=decisions,
                ips_weights=None,
                policy=None),
            'detailed': False
        }
    }
}


if os.path.isfile('./cluster_experiments/seeds.npz'):
    seeds = np.load('./cluster_experiments/seeds.npz')
    training_parameters['data']["training_seeds"] = seeds["train"]
    training_parameters['data']["test_seed"] = seeds["test"]
else:
    seeds = {}
    train_seeds = get_list_of_seeds(200)
    test_seeds = get_list_of_seeds(1)
    training_parameters['data']["training_seeds"] = train_seeds
    training_parameters['data']["test_seed"] = test_seeds
    np.savez('./cluster_experiments/seeds.npz', train=train_seeds, test=test_seeds)

### No Fariness

In [ ]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            lambda **util_params: cost_utility(cost_factor=0.5, **util_params),
                                            lambda **util_params: cost_utility_gradient(cost_factor=0.5, **util_params),
                                            lambda **util_params: 0.0,
                                            lambda **util_params: 0.0)

training_parameters["save_path"] = "../res/local_experiments/ADULT/NO_FAIRNESS"
statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=30,
    asynchronous=True,
    fairness_rates=[0.0])

plot_mean(x_values=lambdas,
          x_label="Penalty Constant",
          x_scale="log",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean.png".format(run_path))
plot_median(x_values=lambdas,
            x_label="Penalty Constant",
            x_scale="log",

          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
            file_path="{}/results_median.png".format(run_path))


### Benefit Difference: DP

In [ ]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            lambda **util_params: cost_utility(cost_factor=0.5, **util_params),
                                            lambda **util_params: cost_utility_gradient(cost_factor=0.5, **util_params),
                                            lambda **util_params: fairness_function("BD_DP", **util_params),
                                            lambda **util_params: fairness_function_gradient("BD_DP", **util_params))


training_parameters["save_path"] = "../res/local_experiments/ADULT_BEST/BD_DP"
# lambdas = np.linspace(0.00001, 0.0001, endpoint=True, num=10).tolist()
# lambdas = np.linspace(0.0001, 0.001, endpoint=True, num=10).tolist()
# lambdas = np.linspace(0.001, 0.01, endpoint=True, num=10).tolist()
# lambdas = np.linspace(0.01, 0.1, endpoint=True, num=10).tolist()
# lambdas = np.sort(np.unique(np.array(lambdas, dtype=float)))
lambdas = np.geomspace(0.00001, 0.1, endpoint=True, num=10)

statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=10,
    asynchronous=True,
    fairness_rates=lambdas)

plot_mean(x_values=lambdas,
          x_label="Penalty Constant",
          x_scale="log",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean.png".format(run_path))
plot_median(x_values=lambdas,
            x_label="Penalty Constant",
            x_scale="log",

          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
            file_path="{}/results_median.png".format(run_path))


### Benefit Difference: EOP

In [ ]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            lambda **util_params: cost_utility(cost_factor=0.5, **util_params),
                                            lambda **util_params: cost_utility_gradient(cost_factor=0.5, **util_params),
                                            lambda **util_params: fairness_function("BD_EOP", **util_params),
                                            lambda **util_params: fairness_function_gradient("BD_EOP", **util_params))


training_parameters["save_path"] = "../res/local_experiments/ADULT_BEST/BD_EOP"
# lambdas = np.linspace(0.00001, 0.0001, endpoint=True, num=10).tolist()
# lambdas = np.linspace(0.0001, 0.001, endpoint=True, num=10).tolist()
# lambdas = np.linspace(0.001, 0.01, endpoint=True, num=10).tolist()
# lambdas = np.linspace(0.01, 0.1, endpoint=True, num=10).tolist()
# lambdas = np.sort(np.unique(np.array(lambdas, dtype=float)))
lambdas = np.geomspace(0.00001, 0.1, endpoint=True, num=10)

statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=10,
    asynchronous=True,
    fairness_rates=lambdas)

plot_mean(x_values=lambdas,
          x_label="Penalty Constant",
          x_scale="log",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean.png".format(run_path))
plot_median(x_values=lambdas,
            x_label="Penalty Constant",
            x_scale="log",

          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
            file_path="{}/results_median.png".format(run_path))

# FICO Credit Score Data

In [ ]:
bias = True
distribution = FICODistribution(bias=bias, fraction_protected=0.5)
dim_theta = distribution.feature_dimension

training_parameters = {
    'model': LogisticPolicy(np.zeros(dim_theta), IdentityFeatureMap(dim_theta), False),
    'distribution': distribution,
    'parameter_optimization': {
        'time_steps':200,
        'epochs': 50,
        'batch_size':128,
        'learning_rate': 0.01,
        'learn_on_entire_history': False,
        'clip_weights': True
    },
    'data': {
        'num_train_samples': 4096,
        'num_test_samples': 1024
    },
    'evaluation': {
        UTILITY: {
            'measure_function': lambda s, y, decisions : cost_utility(cost_factor=0.5,
                                                                      s=s,
                                                                      y=y,
                                                                      decisions=decisions),
            'detailed': False
        },
        COVARIANCE_OF_DECISION_DP: {
            'measure_function': lambda s, y, decisions : fairness_function(
                type="COV_DP",
                x=None,
                s=s,
                y=y,
                decisions=decisions,
                ips_weights=None,
                policy=None),
            'detailed': False
        }
    }
}


if os.path.isfile('./cluster_experiments/seeds.npz'):
    seeds = np.load('./cluster_experiments/seeds.npz')
    training_parameters['data']["training_seeds"] = seeds["train"]
    training_parameters['data']["test_seed"] = seeds["test"]
else:
    seeds = {}
    train_seeds = get_list_of_seeds(200)
    test_seeds = get_list_of_seeds(1)
    training_parameters['data']["training_seeds"] = train_seeds
    training_parameters['data']["test_seed"] = test_seeds
    np.savez('./cluster_experiments/seeds.npz', train=train_seeds, test=test_seeds)

### Benefit Difference: DP

In [ ]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            lambda **util_params: cost_utility(cost_factor=0.5, **util_params),
                                            lambda **util_params: cost_utility_gradient(cost_factor=0.5, **util_params),
                                            lambda **util_params: fairness_function("BD_DP", **util_params),
                                            lambda **util_params: fairness_function_gradient("BD_DP", **util_params))

training_parameters["save_path"] = "../res/local_experiments/FICO_BEST/BD_DP"
# lambdas = np.linspace(0.0001, 0.001, endpoint=True, num=10).tolist()
# lambdas.extend(np.linspace(0.001, 0.01, endpoint=True, num=10).tolist())
# lambdas.extend(np.linspace(0.01, 0.1, endpoint=True, num=10).tolist())
# lambdas = np.sort(np.unique(np.array(lambdas, dtype=float)))
lambdas = np.geomspace(0.00001, 0.1, endpoint=True, num=10)

statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=10,
    asynchronous=True,
    fairness_rates=lambdas)

plot_mean(x_values=lambdas,
          x_label="Penalty Constant",
          x_scale="log",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean.png".format(run_path))
plot_median(x_values=lambdas,
            x_label="Penalty Constant",
            x_scale="log",

          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
            file_path="{}/results_median.png".format(run_path))

### Benefit Difference: EOP

In [ ]:
training_parameters["optimization_target"] = \
    ManualGradientPenaltyOptimizationTarget(0.0,
                                            lambda **util_params: cost_utility(cost_factor=0.5, **util_params),
                                            lambda **util_params: cost_utility_gradient(cost_factor=0.5, **util_params),
                                            lambda **util_params: fairness_function("BD_EOP", **util_params),
                                            lambda **util_params: fairness_function_gradient("BD_EOP", **util_params))


training_parameters["save_path"] = "../res/local_experiments/FICO_BEST/BD_EOP"
# lambdas = np.linspace(0.001, 0.01, endpoint=True, num=10).tolist()
# lambdas.extend(np.linspace(0.01, 0.1, endpoint=True, num=10).tolist())
# lambdas.extend(np.linspace(0.1, 1.0, endpoint=True, num=10).tolist())
# lambdas = np.sort(np.unique(np.array(lambdas, dtype=float)))
lambdas = np.geomspace(0.00001, 1.0, endpoint=True, num=10)

statistics, model_parameters, run_path = train(
    training_parameters,
    iterations=10,
    asynchronous=True,
    fairness_rates=lambdas)

plot_mean(x_values=lambdas,
          x_label="Penalty Constant",
          x_scale="log",
          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
          file_path="{}/results_mean.png".format(run_path))
plot_median(x_values=lambdas,
            x_label="Penalty Constant",
            x_scale="log",

          performance_measures=[statistics.get_additonal_measure(UTILITY, "Utility"),
                                statistics.demographic_parity(),
                                statistics.equality_of_opportunity()],
          fairness_measures=[],
            file_path="{}/results_median.png".format(run_path))